# PIPELINE MULTIPLE SIMULATION-K MEANS CLUSTERING

In [25]:
import json
import sys
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
np.set_printoptions(precision=4)
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# Plotting functions

## Plot the positions of the finger

In [136]:
    def plot_pos_fingers(sides_pos,corners_pos,dim,pose,gap,trial):        
        SMALL_SIZE = 8
        MEDIUM_SIZE = 20
        BIGGER_SIZE = 30

        plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

        fig, ax = plt.subplots()
        plt.style.use("ggplot")
        plt.rcParams["figure.figsize"] = (12,8)
        plt.scatter(sides_pos.T[0],sides_pos.T[2],color='b',marker='+',label='sides')
        plt.scatter(corners_pos.T[0],corners_pos.T[2],color='r',marker='+',label='corners')
        plt.ylabel(r'$\theta_R$')
        plt.xlabel(r'$\theta_L$')
        plt.legend(loc='lower right')
        plt.title("Raw data: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
        plt.savefig('raw_data_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

## Plot projection on feature vectors

In [147]:
def plot_proj_ld1(lda_X,y,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            ax.axes.yaxis.set_ticks([])
            plt.ylabel('LD1')
            plt.scatter(lda_X[y==0], np.full((len(lda_X[y==0]),),1.0), s=100, color='b', label='corner')
            plt.scatter(lda_X[y==1], np.full((len(lda_X[y==1]),),1.0), s=100, color='r', label='side')
            #plt.plot([-1.4,-1.4],[0.9,1.1],'--',color='g')
            plt.ylim(0.9,1.1)
            ax.legend()
            plt.title("1D LDA: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('1D_LDA_proj_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

## 2-means

In [148]:
def plot_ld1_2means(lda_X,y,y_k2means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            ax.axes.yaxis.set_ticks([])
            plt.ylabel('LD1 & 2-means clustering')
            plt.scatter(lda_X[y==0], np.full((len(lda_X[y==0]),),2), color='orange', label='corner')
            plt.scatter(lda_X[y==1], np.full((len(lda_X[y==1]),),2), color='purple', label='side')
            plt.scatter(lda_X[y_k2means==0], np.full((len(lda_X[y_k2means==0]),),1.0), c='red', label ='Cluster 1')
            plt.scatter(lda_X[y_k2means==1], np.full((len(lda_X[y_k2means==1]),),1.0), c='blue', label ='Cluster 2')
            plt.scatter(k2means.cluster_centers_[:, 0], np.full((len(k2means.cluster_centers_[:, 0]),),1.0), 
                        s=100, c='yellow', marker='x',label = 'Centroids')
            #plt.plot([-1.4,-1.4],[0,4],'--',color='g')
            plt.ylim(0,4)
            ax.legend()
            plt.title("LD1 & 2-means: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('LD1_2_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')
            #Plot the centroid. This time we're going to use the cluster centres  #attribute that returns here the coordinates of the centroid.
            
            ##########################################################################################

def plot_raw_2means(sides_pos,corners_pos,angle_array,half_window_size,y_k2means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
            
            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            plt.scatter(sides_pos.T[0],sides_pos.T[2]+10,
                        color='purple',marker='+',label='sides')
            plt.scatter(corners_pos.T[0],corners_pos.T[2]+10,
                        color='orange',marker='+',label='corners')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k2means == 0], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k2means == 0]-10, 
                        marker='+', c='red', label ='Cluster 1')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k2means == 1], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k2means == 1]-10, 
                        marker='+', c='blue', label ='Cluster 2')
            plt.plot(angle_array[half_window_size:-half_window_size].T[0],
                    angle_array[half_window_size:-half_window_size].T[2],
                    color='yellow', label='raw data')
            plt.ylabel(r'$\theta_R$')
            plt.xlabel(r'$\theta_L$')
            plt.legend(loc='lower right')
            plt.title("raw data processed (2-means): dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('clusters_class_raw_2_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

## 3-means

In [149]:
def plot_ld1_3means(lda_X,y,y_k3means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            ax.axes.yaxis.set_ticks([])
            plt.ylabel('LD1 & 3-means clustering')
            plt.scatter(lda_X[y==0], np.full((len(lda_X[y==0]),),2), color='orange', label='corner')
            plt.scatter(lda_X[y==1], np.full((len(lda_X[y==1]),),2), color='purple', label='side')
            plt.scatter(lda_X[y_k3means==0], np.full((len(lda_X[y_k3means==0]),),1.0), c='red', label ='Cluster 1')
            plt.scatter(lda_X[y_k3means==1], np.full((len(lda_X[y_k3means==1]),),1.0), c='blue', label ='Cluster 2')
            plt.scatter(lda_X[y_k3means==2], np.full((len(lda_X[y_k3means==2]),),1.0), c='green', label ='Cluster 3')
            plt.scatter(k3means.cluster_centers_[:, 0], np.full((len(k3means.cluster_centers_[:, 0]),),1.0), 
                        s=100, c='yellow', marker='x',label = 'Centroids')
            #plt.plot([-3,-3],[0,4],'--',color='g')
            plt.ylim(0,4)
            ax.legend()
            plt.title("LD1 & 3-means: dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('LD1_3_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

            ##########################################################################################

def plot_raw_3means(sides_pos,corners_pos,angle_array,half_window_size,y_k3means,dim,pose,gap,trial):
            SMALL_SIZE = 8
            MEDIUM_SIZE = 20
            BIGGER_SIZE = 30

            plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
            
            fig, ax = plt.subplots()
            plt.style.use("ggplot")
            plt.rcParams["figure.figsize"] = (12,8)
            plt.scatter(sides_pos.T[0],sides_pos.T[2]+10,
                        color='purple',marker='+',label='sides')
            plt.scatter(corners_pos.T[0],corners_pos.T[2]+10,
                        color='orange',marker='+',label='corners')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k3means == 0], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k3means == 0]-10, 
                        marker='+', c='red', label ='Cluster 1')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k3means == 1], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k3means == 1]-10, 
                        marker='+', c='blue', label ='Cluster 2')
            plt.scatter(angle_array[half_window_size:-half_window_size].T[0][y_k3means == 2], 
                        angle_array[half_window_size:-half_window_size].T[2][y_k3means == 2]-10, 
                        marker='+', c='green', label ='Cluster 3')
            plt.plot(angle_array[half_window_size:-half_window_size].T[0],
                    angle_array[half_window_size:-half_window_size].T[2],
                    color='yellow', label='raw data')
            plt.ylabel(r'$\theta_R$')
            plt.xlabel(r'$\theta_L$')
            plt.legend(loc='lower right')
            plt.title("raw data processed (3-means): dim="+str(dim)+'x'+str(dim)+ 'mm ; pose='+str(pose)+'mm ; gap='+str(gap)+'mm ; trial='+str(trial),fontsize=20)
            plt.savefig('clusters_class_raw_3_means_dim'+str(dim)+'x'+str(dim)+ '_pose_'+str(pose)+'_gap_'+str(gap)+'_'+str(trial)+'.png')

# Dataset

In [158]:
with open('data_sqrt_trial_classified.json') as f:
    full_data_dict = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
trials = [1,2]
f1_scores = {}

for trial in trials:
    for pose in poses_drf:
        for gap in finger_gaps:
            for dim in sqrt_sides:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                data_dict = full_data_dict[dict_key]
    
                data_list = [data_dict['LF_motion1'],data_dict['RF_motion1']]
                data_array = np.array(data_list).T

                temp_data_array = np.append(data_array[np.newaxis,0,:,0],data_array[np.newaxis,1,:,0],axis=0)
                temp_data_array = np.append(temp_data_array,data_array[np.newaxis,0,:,1],axis=0)
                temp_data_array = np.append(temp_data_array,data_array[np.newaxis,1,:,1],axis=0)
                data_array = np.append(temp_data_array,data_array[np.newaxis,2,:,0],axis=0).T
                angle_array = data_array[:,:4].astype(float)
                # CREATE A DATAFRAME BY WINDOWING THE RAW AND INCREMENTING THE POSITION OF THE WINDOW
                rows,columns = data_array.shape
                window_size = 5 # needs to be odd
                half_window_size = math.floor(window_size/2)
                data_window = np.zeros((rows-half_window_size*2,window_size*4+1)) #+1 for the class
                corners = []
                sides = []
                for i in range(half_window_size,rows-half_window_size):
                    temp_window = np.append(data_array[i-half_window_size:half_window_size+i+1,:].T[0],
                                            data_array[i-half_window_size:half_window_size+i+1,:].T[1])
                    temp_window = np.append(temp_window,
                                            data_array[i-half_window_size:half_window_size+i+1,:].T[2])
                    temp_window = np.append(temp_window,
                                            data_array[i-half_window_size:half_window_size+i+1,:].T[3])
                    if 'corner' in data_array[i-half_window_size+1:half_window_size+i+1-1,:].T[4]:
                        # CLASS 2 IS THE CORNER
                        corners += [angle_array[i].tolist()]
                        data_window[i-half_window_size] = np.append(temp_window,np.array([2]))
                    else:
                        # CLASS 1 IS THE SIDE
                        sides += [angle_array[i].tolist()]
                        data_window[i-half_window_size] = np.append(temp_window,np.array([1]))
    
                #print(data_window)
                names = []
                for j in range(window_size):
                    names += ["angleLF"+str(j+1)]
                for j in range(window_size):
                    names += ["timeLF"+str(j+1)]
                for j in range(window_size):
                    names += ["angleRF"+str(j+1)]
                for j in range(window_size):
                    names += ["timeRF"+str(j+1)]
                names += ["class"]

                # DATA FRAME PANDAS
                df = pd.DataFrame(data=data_window,columns = names)
                df["class"].replace({1.0: "side", 2.0: "corner"}, inplace=True)
                df.dropna(how='all', inplace=True)
                windows = df[names[:len(names)-1]]
                labels = df[names[-1]]

                corners_pos = np.array(corners)
                sides_pos = np.array(sides)

                df.head()
                
                ##########################################################################################
                ############################# Plot the positions of the finger ###########################
                ##########################################################################################
                #plot_pos_fingers(sides_pos,corners_pos,dim,pose,gap,trial)

                ##########################################################################################
                ############################# LDA with sklearn ###########################
                ##########################################################################################

                # Preparation of the data for LDA
                X = df.iloc[:, 0:-1].values
                le = LabelEncoder()
                y = le.fit_transform(df['class'])

                # Import LDA from sklearn
                lda = LinearDiscriminantAnalysis()
                lda_X = lda.fit_transform(X, y)
                print(lda_X)
                print("The explained variance ratio of lda is : ", lda.explained_variance_ratio_)

                ##########################################################################################
                ############################# Plot projection on feature vectors ###########################
                ##########################################################################################
                #plot_proj_ld1(lda_X,y,dim,pose,gap,trial)
                
                ##########################################################################################
                #################################### K-means #############################################
                ##########################################################################################

                # 2-means clustering 
                k2means = KMeans(n_clusters=2, init ='k-means++', max_iter=300, n_init=10,random_state=0 )
                y_k2means = k2means.fit_predict(lda_X)

                # 3-mens clustering
                k3means = KMeans(n_clusters=3, init ='k-means++', max_iter=300, n_init=10,random_state=0 )
                y_k3means = k3means.fit_predict(lda_X)

                ##########################################################################################
                #plot_ld1_2means(lda_X,y,y_k2means,dim,pose,gap,trial)
                ##########################################################################################
                #plot_raw_2means(sides_pos,corners_pos,angle_array,half_window_size,y_k2means,dim,pose,gap,trial)
                ##########################################################################################
                #plot_ld1_3means(lda_X,y,y_k3means,dim,pose,gap,trial)
                ##########################################################################################
                #plot_raw_3means(sides_pos,corners_pos,angle_array,half_window_size,y_k3means,dim,pose,gap,trial)
                ##########################################################################################

                ##########################################################################################
                #################################### F1-score ############################################
                ##########################################################################################

                # we need to change "y" before beca
                y_true = []
                for i in range(y.shape[0]):
                    if y[i] == 0:
                        y_true += [1]
                    else:
                        y_true += [0]

                f1_score_2means = f1_score(y_true,y_k2means)
                print("The f1_score of the 2-means is : ",f1_score_2means)
                print(y_true)

                # f1-score on just the corner identification
                y_k3_corner = []
                for i in range(len(y_k3means.tolist())):
                    if y_k3means[i] == 2:
                        y_k3_corner += [1]
                    else :
                        y_k3_corner += [y_k3means[i]]
                y_k3_corner = np.array(y_k3_corner)

                f1_score_3means = f1_score(y_true,y_k3_corner)
                print("The f1_score of the 3-means for global corner identification is : ",f1_score_3means)

                # f1-score on the number of corners
                u = 0
                v = 0
                y_true_corners = []
                for i in range(len(y_true)):
                    if y_true[i] == 1 and v == 0:
                        y_true_corners += [y_true[i]]
                        u = 1
                    elif y_true[i] == 0 and u==1:
                        y_true_corners += [y_true[i]]
                        v = 1
                    elif y_true[i] == 1 and v==1:
                        y_true_corners += [2]
                    else:
                        y_true_corners += [y_true[i]]
                acc_3means = accuracy_score(y_true_corners,y_k3means)
                print("The f1_score of the 3-means for each corner identification (micro/or accuracy) is : ",
                      acc_3means)

                f1_scores[dict_key] = {
                    "The f1_score 2-means corner identification" : f1_score_2means,
                    "The f1_score 3-means global corner identification" : f1_score_3means,
                    "The f1_score 3-means each corner identification" : acc_3means      
                }
            
print(f1_scores)

[[ 0.1187]
 [ 0.9451]
 [-0.0681]
 [ 1.8491]
 [-0.5594]
 [ 0.7398]
 [ 1.9239]
 [ 1.699 ]
 [ 1.5923]
 [ 0.7476]
 [ 0.4197]
 [ 1.2695]
 [ 2.8174]
 [ 1.8515]
 [ 1.7975]
 [ 1.0068]
 [-1.6314]
 [-1.4343]
 [-0.3685]
 [ 1.0669]
 [ 1.9452]
 [-1.4085]
 [-6.41  ]
 [-7.7026]
 [-8.6448]
 [-7.8366]
 [-6.4806]
 [-4.8658]
 [-0.9304]
 [-0.9084]
 [-1.1038]
 [ 1.0938]
 [ 1.1489]
 [ 1.2234]
 [ 1.3654]
 [ 1.103 ]
 [ 1.3709]
 [ 2.0328]
 [ 0.6112]
 [ 0.4307]
 [ 0.3721]
 [ 0.2665]
 [ 0.4039]
 [ 0.9526]
 [ 1.3038]
 [ 0.8316]
 [ 1.1172]
 [-0.7397]
 [ 0.0929]
 [ 0.3471]
 [ 0.2326]
 [ 0.5447]
 [ 0.7306]
 [ 0.6582]
 [ 0.3174]
 [ 0.4589]
 [ 0.8138]
 [ 0.5863]
 [ 0.7891]
 [ 1.0793]
 [ 1.3264]
 [ 1.8238]
 [ 1.7927]
 [ 2.0813]]
The explained variance ratio of lda is :  [1.]
The f1_score of the 2-means is :  1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of t

The f1_score of the 2-means is :  0.923076923076923
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.4444444444444445
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.734375
[[ 3.3081e-01]
 [-8.8958e-02]
 [ 1.0345e+00]
 [ 1.4002e+00]
 [ 1.5549e+00]
 [-1.3320e-01]
 [-1.5896e-01]
 [ 8.7244e-01]
 [ 2.6038e+00]
 [ 1.6555e+00]
 [ 1.2881e+00]
 [-1.1855e-01]
 [-1.6108e-01]
 [ 6.4146e-01]
 [ 1.2251e+00]
 [-8.5196e-01]
 [-3.7403e-01]
 [ 3.1418e-01]
 [-1.1687e-01]
 [ 9.1371e-01]
 [-9.7370e-01]
 [-5.8482e+00]
 [-6.1107e+00]
 [-5.5324e+00]
 [-1.1298e+00]
 [-2.1087e+00]
 [-3.1740e+00]
 [-2.8178e+00]
 [-1.4197e+00]
 [-1.9946e+00]
 [-1.6067e+00]
 [ 9.8429e-02]
 [-7.5715e-01]
 [ 6.8488e-01]
 [ 1.5127e+00]
 [-1.5788e-01]
 [ 1.2620e+00]
 [ 1.7316e+00]
 [ 9.

The f1_score of the 2-means is :  0.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.7058823529411764
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.921875
[[-0.0569]
 [-0.039 ]
 [-0.0083]
 [ 0.7967]
 [-0.5185]
 [ 2.1849]
 [ 3.1022]
 [ 3.0761]
 [ 1.2195]
 [-0.2765]
 [ 1.462 ]
 [ 1.786 ]
 [ 1.5721]
 [ 0.1386]
 [ 0.178 ]
 [-0.7644]
 [-0.8281]
 [-1.2339]
 [-1.3218]
 [ 0.1041]
 [ 1.2457]
 [-0.2875]
 [-4.0379]
 [-6.4789]
 [-5.6216]
 [-6.2753]
 [-4.2963]
 [-3.2761]
 [-0.6921]
 [-0.9479]
 [ 0.226 ]
 [ 0.9225]
 [ 0.2045]
 [ 0.6996]
 [-0.371 ]
 [-1.0303]
 [ 0.0167]
 [ 0.0261]
 [ 0.8585]
 [ 1.4992]
 [ 0.9595]
 [ 1.5563]
 [ 0.8381]
 [-0.2138]
 [ 0.3765]
 [-0.007 ]
 [-0.2315]
 [-0.0458]
 [ 0.3017]
 [ 1.224 ]
 [ 0.0223]
 [ 1.0649]
 [ 0.4742]
 [ 1.2

The f1_score of the 2-means is :  0.9090909090909091
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.35294117647058826
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.625
[[ 2.4063e-01]
 [-3.7208e-01]
 [ 3.7066e-01]
 [-2.5815e-02]
 [ 6.9164e-01]
 [ 6.6454e-01]
 [ 2.4815e+00]
 [ 2.9198e+00]
 [ 9.4221e-01]
 [ 8.7945e-01]
 [-1.9972e-03]
 [ 7.6247e-01]
 [ 2.2965e+00]
 [ 1.7606e+00]
 [ 1.0498e+00]
 [ 8.6559e-02]
 [-1.3951e+00]
 [-3.6266e-01]
 [-4.5134e-01]
 [ 6.1341e-01]
 [ 8.4897e-02]
 [-1.0246e+00]
 [-4.4724e+00]
 [-5.8474e+00]
 [-7.0385e+00]
 [-6.1391e+00]
 [-4.7063e+00]
 [-2.3729e+00]
 [-1.3116e+00]
 [ 1.0370e+00]
 [ 1.7767e-01]
 [ 4.3123e-02]
 [-6.7107e-01]
 [-1.0734e+00]
 [ 9.0386e-02]
 [ 4.0289e-01]
 [ 1.2325e+00]
 [ 1.3578e+00]
 [ 1.4

The f1_score of the 2-means is :  0.923076923076923
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.24000000000000002
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.359375
[[ 4.1629e-01]
 [ 2.7796e-01]
 [ 7.6807e-01]
 [ 7.7139e-01]
 [-5.6330e-01]
 [-1.0826e-01]
 [ 2.4794e+00]
 [ 1.9515e+00]
 [ 9.9268e-01]
 [ 8.9842e-01]
 [ 6.3559e-01]
 [ 1.6482e+00]
 [ 1.6172e+00]
 [ 1.6859e+00]
 [ 1.0240e+00]
 [ 2.8728e-01]
 [-1.3745e+00]
 [-6.1626e-01]
 [-1.4725e-03]
 [ 7.3724e-01]
 [ 1.0316e+00]
 [-1.1959e+00]
 [-5.3591e+00]
 [-6.9434e+00]
 [-7.0936e+00]
 [-5.2881e+00]
 [-3.9506e+00]
 [-2.1180e+00]
 [-1.0841e+00]
 [-4.7398e-01]
 [-1.7161e+00]
 [-5.3014e-01]
 [ 7.0230e-01]
 [ 7.6818e-01]
 [ 4.6321e-01]
 [ 8.8692e-01]
 [ 8.5742e-01]
 [ 5.7769e-01]
 [-2

The f1_score of the 2-means is :  0.923076923076923
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.8571428571428571
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.96875
[[ 5.0221e-01]
 [ 1.8125e+00]
 [ 2.2663e+00]
 [ 1.4459e+00]
 [ 1.0930e+00]
 [-8.0811e-01]
 [-1.0521e+00]
 [ 6.2623e-01]
 [ 2.2061e+00]
 [ 9.8541e-01]
 [-1.8129e+00]
 [ 3.3318e-01]
 [ 1.4533e+00]
 [ 1.4035e+00]
 [ 9.5294e-01]
 [ 4.4591e-01]
 [-6.5620e-01]
 [ 9.5031e-02]
 [ 5.9082e-01]
 [ 8.2747e-01]
 [-1.2304e+00]
 [-5.5531e+00]
 [-5.9375e+00]
 [-6.2326e+00]
 [-1.5638e+00]
 [-4.0421e+00]
 [-4.2802e+00]
 [-4.0055e+00]
 [-1.8434e+00]
 [-6.4574e-01]
 [-1.3607e+00]
 [ 7.3544e-01]
 [ 7.5128e-01]
 [-4.1362e-01]
 [ 1.4073e-01]
 [ 1.8121e+00]
 [-8.7697e-03]
 [ 2.3490e+00]
 [ 1.5

The f1_score of the 2-means is :  1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The f1_score of the 3-means for global corner identification is :  0.23529411764705882
The f1_score of the 3-means for each corner identification (micro/or accuracy) is :  0.34375
[[ 0.3135]
 [ 2.0039]
 [ 0.2709]
 [-0.7863]
 [ 0.5268]
 [ 0.4803]
 [ 1.7079]
 [ 1.8382]
 [ 3.1717]
 [ 1.3703]
 [-0.3618]
 [ 1.4498]
 [-0.5422]
 [ 0.1004]
 [-0.167 ]
 [-0.8337]
 [-0.9681]
 [-0.0668]
 [-0.467 ]
 [-0.9688]
 [-2.7721]
 [-6.3771]
 [-5.1234]
 [-0.9944]
 [-3.9296]
 [-5.8138]
 [-2.6389]
 [-0.6018]
 [-2.5825]
 [-0.291 ]
 [ 0.2291]
 [-0.5237]
 [ 1.3438]
 [ 0.7737]
 [ 1.6877]
 [ 0.8661]
 [ 0.0259]
 [ 0.4425]
 [ 0.4663]
 [ 0.6994]
 [ 1.0843]
 [ 0.206 ]
 [ 1.0712]
 [ 1.4105]
 [ 0.3548]
 [ 0.987 ]
 [ 0.3696]
 [ 0.6904]
 [ 0.6942]
 [ 0.8227]
 [ 0.2672]
 [ 0.5422]
 [ 0.2921]
 [ 0.4

In [152]:
with open("f1_scores_sqrt_dim_new_w_5_trials.json", 'w') as f:
    indent = 2  # is not needed but makes the file human-readable
    json.dump(f1_scores, f, indent=2)

# Create a table with the result

In [155]:
with open('f1_scores_sqrt_dim_new_w_5_trials.json') as f:
    dict_f1 = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
trials = [1,2]

print("\nTABLE 1 \n")
# Print the names of the columns.
print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format('DIMENSION', 'POSE', 'GAP', 'F1 2-Means','F1 3-Means','F1 3-Means (each)'))

# print each data item.
for gap in finger_gaps:
    for pose in poses_drf:
        for dim in sqrt_sides:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(dim,pose,gap,pose,f1_2means, f1_3means_glob, f1_3means_each))


print("\nTABLE 2 \n")
# Print the names of the columns.
print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format('DIMENSION', 'POSE', 'GAP', 'F1 2-Means','F1 3-Means','F1 3-Means (each)'))

for pose in poses_drf:
    for gap in finger_gaps:
        for dim in sqrt_sides:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(dim,pose,gap,pose,f1_2means, f1_3means_glob, f1_3means_each))


print("\nTABLE 3 \n")
# Print the names of the columns.
print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format('DIMENSION', 'POSE', 'GAP', 'F1 2-Means','F1 3-Means','F1 3-Means (each)'))

for dim in sqrt_sides:
    for gap in finger_gaps:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                print ("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(dim,pose,gap,pose,f1_2means, f1_3means_glob, f1_3means_each))




TABLE 1 

DIMENSION    POSE         GAP          F1 2-Means   F1 3-Means   F1 3-Means (each)
20           60           34           60           1.0          0.364       
20           60           34           60           0.909        0.256       
25           60           34           60           1.0          0.145       
25           60           34           60           0.889        0.2         
30           60           34           60           0.769        0.455       
30           60           34           60           0.833        0.357       
20           65           34           65           1.0          0.273       
20           65           34           65           0.909        0.429       
25           65           34           65           1.0          0.3         
25           65           34           65           0.909        0.286       
30           65           34           65           0.923        0.308       
30           65           34           65       

# Calculate the average f1-score

In [156]:
with open('f1_scores_sqrt_dim_new_w_5_trials.json') as f:
    dict_f1 = json.load(f)

poses_drf = [60,65,70]
finger_gaps = [34,42,50]
sqrt_sides = [20,25,30]
trials = [1,2]

path = 'f1_scores_sqrt_dim_new_w_5_trials.txt'
f1_file = open(path,'w')

title = 'All the F1-score and accuracy of the squares simulation for window of size 5 \n'
f1_file.write(title)


################ same size ################ 
f1_file.write('\nSAME SIZE \n\n')
print('\nSAME SIZE \n')

sum_f1_score_2means = 0
sum_f1_score_3means_glob = 0
sum_f1_score_3means_each = 0
nb_f1_Scores = 0

for dim in sqrt_sides:
    for gap in finger_gaps:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
    print('The average f1-score for squares of dim =',str(dim),' and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
    f1_file.write('The average f1-score for squares of dim ='+str(dim)+' and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
    print('The average f1-score for squares of dim =',str(dim),' and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
    f1_file.write('The average f1-score for squares of dim ='+str(dim)+' and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
    print('The average accuracy for squares of dim =',str(dim),' and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
    f1_file.write('The average accuracy for squares of dim ='+str(dim)+' and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')
    sum_f1_score_2means = 0
    sum_f1_score_3means_glob = 0
    sum_f1_score_3means_each = 0
    nb_f1_Scores = 0
    

################ same pose ################ 
print('\nSAME POSE \n')
f1_file.write('\nSAME POSE \n\n')
sum_f1_score_2means = 0
sum_f1_score_2means_glob = 0
sum_f1_score_2means_each = 0
nb_f1_Scores = 0
for pose in poses_drf:
    for dim in sqrt_sides:
        for gap in finger_gaps:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
    print('The average f1-score for squares with pose =',str(pose),' and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with pose ='+str(pose)+' and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
    print('The average f1-score for squares with pose =',str(pose),' and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with pose ='+str(pose)+' and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
    print('The average accuracy for squares with pose =',str(pose),' and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
    f1_file.write('The average accuracy for squares with pose ='+str(pose)+' and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')
    sum_f1_score_2means = 0
    sum_f1_score_3means_glob = 0
    sum_f1_score_3means_each = 0
    nb_f1_Scores = 0


################ same gap ################ 
f1_file.write('\nSAME GAP \n\n')
print('\nSAME GAP \n\n')
sum_f1_score_2means = 0
sum_f1_score_2means_glob = 0
sum_f1_score_2means_each = 0
nb_f1_Scores = 0
for gap in finger_gaps:
    for dim in sqrt_sides:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
    print('The average f1-score for squares with fingers gap =',str(gap),' and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with fingers gap ='+str(gap)+' and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
    print('The average f1-score for squares with fingers gap =',str(gap),' and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
    f1_file.write('The average f1-score for squares with fingers gap ='+str(gap)+' and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
    print('The average accuracy for squares with fingers gap =',str(gap),' and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
    f1_file.write('The average accuracy for squares with fingers gap ='+str(gap)+' and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')
    sum_f1_score_2means = 0
    sum_f1_score_3means_glob = 0
    sum_f1_score_3means_each = 0
    nb_f1_Scores = 0

################ total average ################ 
f1_file.write('\nTOTAL AVERAGE \n\n')
print('\nTOTAL AVERAGE \n\n')
sum_f1_score_2means = 0
sum_f1_score_3means_glob = 0
sum_f1_score_3means_each = 0
nb_f1_Scores = 0
for gap in finger_gaps:
    for dim in sqrt_sides:
        for pose in poses_drf:
            for trial in trials:
                dict_key = "dim_"+str(dim)+"_pose_"+str(pose)+"_gap_"+str(gap)+'_'+str(trial)
                dict_value = dict_f1[dict_key]
                f1_2means = round(dict_value['The f1_score 2-means corner identification'],3)
                f1_3means_glob = round(dict_value['The f1_score 3-means global corner identification'],3)
                f1_3means_each = round(dict_value['The f1_score 3-means each corner identification'],3)
                sum_f1_score_2means += f1_2means
                sum_f1_score_3means_glob += f1_3means_glob
                sum_f1_score_3means_each += f1_3means_each
                nb_f1_Scores += 1
print('The total average f1-score for squares and 2-means clustering : ',sum_f1_score_2means/nb_f1_Scores)
f1_file.write('The total average f1-score for squares and 2-means clustering : '+str(sum_f1_score_2means/nb_f1_Scores)+'\n')
print('The total average f1-score for squares and 3-means clustering : ',sum_f1_score_3means_glob/nb_f1_Scores)
f1_file.write('The total average f1-score for squares and 3-means clustering : '+str(sum_f1_score_3means_glob/nb_f1_Scores)+'\n')
print('The total average accuracy for squares and 3-means clustering : ',sum_f1_score_3means_each/nb_f1_Scores,'\n')
f1_file.write('The total average accuracy for squares and 3-means clustering : '+str(sum_f1_score_3means_each/nb_f1_Scores)+'\n\n')

f1_file.close()


SAME SIZE 

The average f1-score for squares of dim = 20  and 2-means clustering :  0.9133333333333332
The average f1-score for squares of dim = 20  and 3-means clustering :  0.31555555555555553
The average accuracy for squares of dim = 20  and 3-means clustering :  0.5286666666666667 

The average f1-score for squares of dim = 25  and 2-means clustering :  0.8748888888888889
The average f1-score for squares of dim = 25  and 3-means clustering :  0.30983333333333335
The average accuracy for squares of dim = 25  and 3-means clustering :  0.4835 

The average f1-score for squares of dim = 30  and 2-means clustering :  0.7018333333333333
The average f1-score for squares of dim = 30  and 3-means clustering :  0.3192777777777778
The average accuracy for squares of dim = 30  and 3-means clustering :  0.5442222222222223 


SAME POSE 

The average f1-score for squares with pose = 60  and 2-means clustering :  0.9027222222222223
The average f1-score for squares with pose = 60  and 3-means clus

- poor performance for the small square for the 3-means as the window of are overlapping 
- for gap 42 problems

